# 2.2.3.	Search Case Study - Edge Detection and Depth to Source#

Here we apply the tilt-depth method discussed in Notebook 2.1.4. to estimate depth to source in the Search Phase II magnetic data. Depth can be accurately derived from the tilt angle where contacts between magnetically contrasted geological units are vertical. Because of the typically complex nature of true geological contacts, calculated depths calculated from real magnetic data sets are usually approximations. The results should always be considered in light of expected geology, and ideally should be interpreted alongside geology, topographic data, and other geophysical data and models if these data are available.

<!-- <img src="./images/SearchQuestII.png"> -->


## Total magnetic intensity and area selection

Below is the gridded total magnetic intensity data from the Search Phase II project. The magnetic data is interpolated onto a 50 $m^2$ grid, plotted using coordinate system NAD 83 UTM Zone 9 (EPSG: 3156).  

A localized area or feature of interest can be windowed out for investigation using the tilt-depth method. The data 'window' shown in the left map (black outline) can be resized and moved anywhere over the data using the slider bars.


In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Simulator, DataIO
dataFile = DataIO.fetchData(file=["./assets/Search/MAG_UTM09.tiff"], dtype='GeoTiff', loadFile=True)
display(dataFile)

interactive(children=(RadioButtons(description='File Type:', options=('Local', 'Cloud'), value='Local'), Text(value='./assets/Search/', description='Path:'), ToggleButton(value=False, description='Download', icon='check', tooltip='Description'), ToggleButton(value=False, description='Check folder', icon='check', tooltip='Description'), Dropdown(description='Files:', options=('./assets/Search/MAG_UTM09.tiff',), value='./assets/Search/MAG_UTM09.tiff'), RadioButtons(description='File Type:', index=1, options=('CSV', 'GeoTiff', 'GRD'), value='GeoTiff'), ToggleButton(value=True, description='Load File', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))

In [2]:
selection = Simulator.setDataExtentWidget(dataFile.result[0], East=669500, North=6069500)
display(selection)

interactive(children=(FloatSlider(value=669500.0, continuous_update=False, description='East', max=811100.0, min=578550.0, step=500.0), FloatSlider(value=6069500.0, continuous_update=False, description='North', max=6150300.0, min=5901050.0, step=10.0), FloatSlider(value=30000.0, continuous_update=False, description='Width', max=100000.0, min=1000.0, step=1000.0), FloatSlider(value=30000.0, continuous_update=False, description='Height', max=100000.0, min=1000.0, step=1000.0), Output()), _dom_classes=('widget-interact',))

In [3]:
dataGrid = Simulator.dataGridGeoref(selection.result, EPSGcode=26909, inc=73, dec=20, applyRTP=True)
display(dataGrid)

interactive(children=(Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), FloatText(value=26909.0, description='EPSG code:'), FloatText(value=73.0, description='Inclination angle positive downward from horizontal:'), FloatText(value=20.0, description='Declination angle positve clockwise from North:'), ToggleButton(value=False, description='Fetch Inc/Dec', icon='check', tooltip='Connect to NOAA API'), ToggleButton(value=True, description='Reduce to pole', icon='check', tooltip='Transform to RTP data'), Output()), _dom_classes=('widget-interact',))

## Estimate Depth to Source

**For GoogleEarth, export with EPSG=4326**

In [4]:
view = Simulator.gridTilt2Depth(dataGrid.result, ColorMap='Greys_r')
display(view)

HBox(children=(VBox(children=(FloatSlider(value=90.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.9, continuous_update=False, description='ColorTransp', max=1.0, step=0.05), FloatSlider(value=0.5, continuous_update=False, description='HSTransp', max=1.0, step=0.05), FloatSlider(value=5.0, continuous_update=False, description='vScale', max=10.0, min=1.0, step=1.0), Dropdown(description='ColorMap', index=4, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='Greys_r'), Dropdown(description='Grid Filters', index=1, options=('TMI', 'tiltAngle'), value='tiltAngle'), FloatSlider(value=0.0, continuous_update=False, description='UpC Height', max=200.0, step=10.0), Text(value='./Output/MyGeoTiff', description='GeoTiff name:'), ToggleButton(value=False, description='Export GeoTiff', icon='check', tooltip='Description'), Dropdown(description='Depth Color', options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='viridis'), FloatSlider(value=1.0, continuous_update=False, description='Marker Size', max=10.0, min=0.1), Text(value='./Output/EstimatedDepth', description='Shapefile name:'), ToggleButton(value=False, description='Export Shapefile', icon='check', tooltip='Description')), layout=Layout(margin='0px 0px 0px 0px', width='35%')), VBox(children=(Output(),), layout=Layout(height='800px', margin='0px 0px 0px 0px', width='65%'))))